In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf

seed = 2023
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.experimental.numpy.random.seed(seed)
os.environ["TF_CUDNN_DETERMINISTIC"] = "1"
os.environ["TF_DETERMINISTIC_OPS"] = "1"
# Set a fixed value for the hash seed
os.environ["PYTHONHASHSEED"] = str(seed)
import dolfin as df
import time
from utils import *
from utils_training import *
import prepare_data
from utils_compare_methods import *
import seaborn as sns
from mpl_toolkits.axes_grid1 import make_axes_locatable
import vedo
import vedo.dolfin as vdf
from prepare_data import rotate, outside_ball
import pandas as pd

sns.set_theme()
sns.set_context("paper")
colors = sns.color_palette("mako").as_hex()
my_cmap = sns.color_palette("viridis", as_cmap=True)


In [ ]:
nb_data = 1500
save_figs = True

small_data = False
level = 2
data = DataLoader(small_data)
agent = Agent(data, small_data)

if not (os.path.exists(f"./images_{nb_data}/")) and save_figs:
    os.makedirs(f"./images_{nb_data}/")

epochs = [50, 100, 200, 400, 500, 1000, 1500, 2000]
print(len(epochs))
print(epochs)

indices = list(range(0, len(epochs)))
size_per_fig = 4
nb_rows = 2
size_row = int(len(indices) / nb_rows)


In [ ]:
nb_epochs = 2000
agent.model.load_weights(f"./models_{nb_data}/model_{nb_epochs}/model_weights")


In [ ]:
def change_size_fno(param, sizes, Plot=False, nb_epochs=2000):
    solveur_standard = StandardFEMSolver(params=param)
    u_ref, V_ref, dx_ref = solveur_standard.solve_one(0, None, True)

    errors, sols = [], []
    mu0, mu1, sigma, x_0, y_0, lx, ly, theta, alpha, beta = param[0]

    agent.model.load_weights(
        f"./models_{nb_data}/model_{nb_epochs}/model_weights"
    )

    for size in sizes:
        print(f"{size=}")
        phi = generate_phi_numpy(x_0, y_0, lx, ly, theta, size)
        F = generate_F_numpy(mu0, mu1, sigma, size) / data.max_norm_F
        G = generate_G_numpy(alpha, beta, size)
        X = generate_manual_new_data_numpy(phi, F, G)
        Y = agent.model.call(X)
        predicted_solution = X[:, :, :, 1] * Y[:, :, :, 0] + X[:, :, :, 2]
        predicted_solution = np.reshape(predicted_solution, (size, size))
        predicted_solution_fenics = convert_numpy_matrix_to_fenics(
            predicted_solution, size, 1
        )
        predicted_solution_fenics_proj_V_ex = df.project(
            predicted_solution_fenics,
            V_ref,
            solver_type="gmres",
            preconditioner_type="hypre_amg",
        )
        sols.append(predicted_solution_fenics_proj_V_ex)
        l2_error_fno = (
            df.assemble(
                (((u_ref - predicted_solution_fenics_proj_V_ex)) ** 2) * dx_ref
            )
            ** (0.5)
        ) / (df.assemble((((u_ref)) ** 2) * dx_ref) ** (0.5))
        errors.append(l2_error_fno)

        if Plot:
            plt.figure(figsize=(20, 5.5))
            plt.subplot(1, 4, 1)
            p = df.plot(u_ref, mode="color", cmap=my_cmap)
            plt.colorbar(p)
            plt.title("Reference solution", fontsize="14")
            for i in range(1, 4):
                plt.subplot(1, 4, i + 1)
                p = df.plot(sols[i - 1], mode="color", cmap=my_cmap)
                plt.colorbar(p)
                plt.title(
                    "FNO prediction for nb_vert = "
                    + str(sizes[i - 1])
                    + "\n$L^2$ relative error : "
                    + f"{errors[i-1]}",
                    fontsize="14",
                )

            plt.tight_layout()
            if Plot and save_figs:
                plt.savefig(f"./images_{nb_data}/change_sizes.png")
            plt.show()

    return errors

In [ ]:
errors = []
nb_params_to_test = 5
F, phi, G, params = new_create_FG_numpy(nb_params_to_test, 64)
sizes = [48, 64, 80, 100, 120, 200]
print(f"{len(sizes)=}")
for i in range(np.shape(params)[0]):
    print(f"Param : {i+1}/{np.shape(params)[0]}")
    error = change_size_fno(np.array([params[i]]), sizes, Plot=False)
    errors.append(error)
errors = np.array(errors)


In [ ]:
plt.figure(figsize=(8, 8))

for i in range(np.shape(errors)[1]):
    sns.histplot(
        data=errors[:, i],
        kde=True,
        bins=20,
        edgecolor="k",
        log_scale=True,
        label=f"FNO nb_vert = {sizes[i]}",
        stat="proportion",
        legend=True,
    )
# plt.xlim(1e-4, 1e0)
# plt.ylim(0.0, 0.30)
plt.xlabel("$L^2$ relative error")
plt.legend()

plt.tight_layout()
save_figs = False
if save_figs:
    plt.savefig(f"./images_{nb_data}/histograms_change_sizes.png")
plt.show()


In [ ]:
abs_str = sizes
dataframe = pd.DataFrame(errors, columns=abs_str)
save_figs = True
plt.figure(figsize=(8, 5))

sns.boxplot(data=dataframe, palette="ch:s=-.05,rot=0.06,dark=0.4")
plt.yscale("log")
plt.xlabel("Number of vertices", fontsize=18)
plt.ylabel("$L^2$ relative error", fontsize=18)
plt.tight_layout()
if save_figs:
    plt.savefig(f"./images_{nb_data}/boxplots_new_data_change_size.png")
plt.show()

In [ ]:
errors = []
nb_params_to_test = 1
F, phi, G, params = new_create_FG_numpy(nb_params_to_test, 64)
sizes = [48, 64, 80, 100, 120, 200]
print(list(params[0]))


In [ ]:
domains = []
solutions = []
for size in sizes:
    i = 0
    print(f"Param : {i+1}/{np.shape(params)[0]}")
    mu0, mu1, sigma, x_0, y_0, lx, ly, theta, alpha, beta = (
        0.30122539,
        0.29640635,
        0.25481616,
        0.46192393,
        0.48159562,
        0.37990447,
        0.32648597,
        0.41180092,
        0.37976006,
        -0.31339751,
    )
    phi = generate_phi_numpy(x_0, y_0, lx, ly, theta, size)
    F = generate_F_numpy(mu0, mu1, sigma, size) / data.max_norm_F
    G = generate_G_numpy(alpha, beta, size)
    X = generate_manual_new_data_numpy(phi, F, G)
    Y = agent.model.call(X)
    predicted_solution = X[0, :, :, 1] * Y[0, :, :, 0] + X[0, :, :, 2]
    solutions.append(predicted_solution)
    domains.append(X[:, :, :, -1])

In [ ]:
fig, axes = plt.subplots(figsize=(10, 5), nrows=2, ncols=3)
i = 0
for ax in axes.flat:
    domains_nan = domains[i].numpy()
    domains_nan[domains_nan == 0] = np.nan
    im = ax.imshow(
        (domains_nan * solutions[i])[0, :, :], cmap="viridis", origin="lower"
    )
    if i < 3:
        ax.set_title(f"{sizes[i]} vertices", fontsize=18)
    else:
        ax.set_title(f"{sizes[i]} vertices", fontsize=18, y=-0.2, pad=-14)
    ax.grid(False)

    i += 1
fig.subplots_adjust(right=0.84)
cbar_ax = fig.add_axes([0.84, 0.14, 0.030, 0.7])
fig.colorbar(im, cax=cbar_ax)
plt.savefig(f"./images_{nb_data}/outputs_various_sizes.png")
plt.show()
